# Backtesting Cost Models

This notebook demonstrates the cost model framework for realistic backtesting simulation.

**Status:** Infrastructure complete (Sprint 4, E3-T1)  
**Integration:** Pending full adapter integration

## Overview

Cost models allow you to simulate realistic trading costs:
- **Commission**: Brokerage fees
- **Spread**: Bid-ask spread costs
- **Slippage**: Market impact and execution slippage
- **Borrow**: Margin/leverage costs (future)

All models are **pluggable** and **configurable**. Default models match current backtest behavior (minimal costs).

In [ ]:
# Imports
import pandas as pd

from finbot.core.contracts.costs import CostEvent, CostSummary, CostType
from finbot.services.backtesting.costs import (
    FixedSlippage,
    FixedSpread,
    FlatCommission,
    PercentageCommission,
    ZeroCommission,
    ZeroSlippage,
    ZeroSpread,
)
from finbot.services.backtesting.costs.slippage import SqrtSlippage

## 1. Commission Models

### Zero Commission (Free Trading)

Use this for modern zero-commission brokers or when testing strategies without transaction costs.

In [ ]:
model = ZeroCommission()

# Example trade: Buy 100 shares at $500
cost = model.calculate_cost(
    symbol="SPY",
    quantity=100,
    price=500.0,
    timestamp=pd.Timestamp("2024-01-01")
)

print(f"Model: {model.get_name()}")
print("Trade: 100 shares of SPY at $500")
print(f"Commission: ${cost:.2f}")

### Flat Commission (Per-Share)

Typical for discount brokers: $0.001 per share (e.g., Interactive Brokers).

In [ ]:
model = FlatCommission(per_share=0.001, min_commission=1.0)

# Example trades
trades = [
    (100, 500.0, "Small trade"),
    (1000, 500.0, "Medium trade"),
    (10, 500.0, "Very small trade (min applies)"),
]

print(f"Model: {model.get_name()}\n")
for quantity, price, desc in trades:
    cost = model.calculate_cost("SPY", quantity, price, pd.Timestamp("2024-01-01"))
    print(f"{desc}:")
    print(f"  {quantity} shares × ${price} = ${quantity * price:,.0f} trade value")
    print(f"  Commission: ${cost:.2f}\n")

### Percentage Commission

Some brokers charge a percentage of trade value with min/max bounds.

In [ ]:
model = PercentageCommission(rate=0.001, min_commission=1.0, max_commission=50.0)

# Example trades of different sizes
trades = [
    (10, 50.0, "Very small (min applies)"),
    (100, 500.0, "Medium (percentage applies)"),
    (10000, 100.0, "Large (max applies)"),
]

print(f"Model: {model.get_name()}\n")
for quantity, price, desc in trades:
    cost = model.calculate_cost("SPY", quantity, price, pd.Timestamp("2024-01-01"))
    trade_value = quantity * price
    pct_cost = trade_value * 0.001
    print(f"{desc}:")
    print(f"  Trade value: ${trade_value:,.0f}")
    print(f"  Calculated (0.1%): ${pct_cost:.2f}")
    print(f"  Actual (with min/max): ${cost:.2f}\n")

## 2. Spread Models

### Zero Spread (Mid-Price Execution)

Assumes trades execute at the quoted price with no spread cost. This is the current backtest default.

In [ ]:
model = ZeroSpread()

cost = model.calculate_cost("SPY", 100, 500.0, pd.Timestamp("2024-01-01"))
print(f"Model: {model.get_name()}")
print(f"Spread cost: ${cost:.2f}")

### Fixed Spread

Applies a constant spread cost. Typical values:
- **Large cap ETFs (SPY)**: 1-2 bps
- **Mid cap stocks**: 5-10 bps
- **Small cap stocks**: 20-50 bps

In [ ]:
# Example: 10 bps spread (0.10%)
model = FixedSpread(bps=10)

trade_value = 100 * 500.0  # $50,000
cost = model.calculate_cost("SPY", 100, 500.0, pd.Timestamp("2024-01-01"))

print(f"Model: {model.get_name()}")
print(f"Trade value: ${trade_value:,.0f}")
print(f"Spread cost (one-way): ${cost:.2f}")
print(f"Round-trip spread cost: ${cost * 2:.2f}")
print("\nInterpretation: You pay half the spread when buying (ask) or selling (bid)")

## 3. Slippage Models

### Zero Slippage (Perfect Execution)

Assumes perfect execution at the expected price. Current backtest default.

In [ ]:
model = ZeroSlippage()

cost = model.calculate_cost("SPY", 100, 500.0, pd.Timestamp("2024-01-01"))
print(f"Model: {model.get_name()}")
print(f"Slippage cost: ${cost:.2f}")

### Fixed Slippage

Constant slippage cost as a percentage of trade value. Typical values:
- **Market orders (liquid)**: 2-5 bps
- **Market orders (illiquid)**: 10-30 bps
- **Large orders**: 20-100 bps

In [ ]:
# Example: 5 bps slippage (0.05%)
model = FixedSlippage(bps=5)

trade_value = 100 * 500.0  # $50,000
cost = model.calculate_cost("SPY", 100, 500.0, pd.Timestamp("2024-01-01"))

print(f"Model: {model.get_name()}")
print(f"Trade value: ${trade_value:,.0f}")
print(f"Slippage cost: ${cost:.2f}")
print(f"Slippage as % of trade: {cost / trade_value:.4%}")

### Square-Root Market Impact

Research-based model where impact scales with √(order size). Based on Almgren & Chriss (2001).

Only applies when order size exceeds a threshold fraction of daily volume.

In [ ]:
model = SqrtSlippage(coefficient=0.1, adv_fraction_threshold=0.01)

# Scenario: Different order sizes vs daily volume
scenarios = [
    (100, 100000, "Small order (0.1% of ADV - below threshold)"),
    (2000, 100000, "Medium order (2% of ADV - above threshold)"),
    (5000, 100000, "Large order (5% of ADV - significant impact)"),
]

print(f"Model: {model.get_name()}\n")
for quantity, daily_volume, desc in scenarios:
    cost = model.calculate_cost(
        "SPY", quantity, 500.0, pd.Timestamp("2024-01-01"),
        daily_volume=daily_volume
    )
    trade_value = quantity * 500.0
    adv_fraction = quantity / daily_volume
    
    print(f"{desc}:")
    print(f"  Order size: {quantity:,} shares")
    print(f"  ADV fraction: {adv_fraction:.2%}")
    print(f"  Trade value: ${trade_value:,.0f}")
    print(f"  Market impact: ${cost:.2f}")
    print(f"  Impact as % of trade: {cost / trade_value:.4%}\n")

## 4. Total Cost Analysis

Combining multiple cost types for realistic simulation.

In [ ]:
# Define cost models
commission_model = FlatCommission(per_share=0.001, min_commission=1.0)
spread_model = FixedSpread(bps=10)  # 10 bps = 0.10%
slippage_model = FixedSlippage(bps=5)  # 5 bps = 0.05%

# Example trade
quantity = 1000
price = 500.0
trade_value = quantity * price

# Calculate each cost type
commission = commission_model.calculate_cost("SPY", quantity, price, pd.Timestamp("2024-01-01"))
spread = spread_model.calculate_cost("SPY", quantity, price, pd.Timestamp("2024-01-01"))
slippage = slippage_model.calculate_cost("SPY", quantity, price, pd.Timestamp("2024-01-01"))

total_cost = commission + spread + slippage

print("Total Cost Breakdown")
print("=" * 50)
print(f"Trade: {quantity} shares of SPY at ${price}")
print(f"Trade value: ${trade_value:,.0f}\n")

print("Cost Components:")
print(f"  Commission: ${commission:,.2f}  ({commission/trade_value:.4%} of trade)")
print(f"  Spread:     ${spread:,.2f}  ({spread/trade_value:.4%} of trade)")
print(f"  Slippage:   ${slippage:,.2f}  ({slippage/trade_value:.4%} of trade)")
print(f"  {'-' * 48}")
print(f"  Total:      ${total_cost:,.2f}  ({total_cost/trade_value:.4%} of trade)\n")

print(f"Net proceeds after costs: ${trade_value - total_cost:,.2f}")

# Round-trip cost (buy + sell)
round_trip_cost = total_cost * 2
print(f"\nRound-trip cost (buy + sell): ${round_trip_cost:,.2f} ({round_trip_cost/trade_value:.4%})")

## 5. Cost Summary Aggregation

Tracking costs across multiple trades.

In [ ]:
# Simulate multiple trades
trades = [
    ("SPY", 1000, 500.0),
    ("SPY", -500, 505.0),  # Sell
    ("TLT", 2000, 100.0),
    ("QQQ", 300, 400.0),
]

# Track all cost events
cost_events = []
total_commission = 0.0
total_spread = 0.0
total_slippage = 0.0

timestamp = pd.Timestamp("2024-01-01")

for symbol, qty, price in trades:
    # Calculate costs
    comm = commission_model.calculate_cost(symbol, qty, price, timestamp)
    spr = spread_model.calculate_cost(symbol, qty, price, timestamp)
    slip = slippage_model.calculate_cost(symbol, qty, price, timestamp)
    
    # Track events
    cost_events.append(CostEvent(timestamp, symbol, CostType.COMMISSION, comm, "FlatCommission"))
    cost_events.append(CostEvent(timestamp, symbol, CostType.SPREAD, spr, "FixedSpread(10bps)"))
    cost_events.append(CostEvent(timestamp, symbol, CostType.SLIPPAGE, slip, "FixedSlippage(5bps)"))
    
    total_commission += comm
    total_spread += spr
    total_slippage += slip

# Create cost summary
summary = CostSummary(
    total_commission=total_commission,
    total_spread=total_spread,
    total_slippage=total_slippage,
    total_borrow=0.0,
    total_market_impact=0.0,
    cost_events=tuple(cost_events),
)

print("Multi-Trade Cost Summary")
print("=" * 50)
print(f"Total trades: {len(trades)}\n")

print("Costs by Type:")
for cost_type, amount in summary.costs_by_type().items():
    print(f"  {cost_type.value.title():15} ${amount:,.2f}")
print(f"  {'-' * 35}")
print(f"  {'Total':15} ${summary.total_costs:,.2f}\n")

print("Costs by Symbol:")
for symbol, amount in sorted(summary.costs_by_symbol().items()):
    print(f"  {symbol:10} ${amount:,.2f}")

## 6. Future Integration

**Current Status:** Cost models are implemented and tested, but not yet integrated into the backtest adapter.

**Upcoming Work:**
- Add `CostSummary` to `BacktestRunResult` schema
- Configure cost models in `BacktraderAdapter`
- Track cost events during backtest execution
- Display cost breakdowns in results

**Usage Pattern (Future):**
```python
from finbot.services.backtesting.adapters.backtrader_adapter import BacktraderAdapter
from finbot.services.backtesting.costs import FlatCommission, FixedSpread

adapter = BacktraderAdapter(
    price_histories=histories,
    commission_model=FlatCommission(per_share=0.001),
    spread_model=FixedSpread(bps=10),
    slippage_model=FixedSlippage(bps=5),
)

result = adapter.run(request)
print(f"Total costs: ${result.costs.total_costs:.2f}")
```

**Note:** Default models will match current behavior (minimal costs) to maintain parity with existing backtests.

## References

1. **Market Impact Models:**
   - Almgren, R., & Chriss, N. (2001). "Optimal execution of portfolio transactions." Journal of Risk, 3, 5-40.
   
2. **Transaction Cost Analysis:**
   - Kissell, R., & Glantz, M. (2013). "Optimal Trading Strategies." AMACOM.
   
3. **Cost Model Documentation:**
   - See `docs/planning/epic-e3-fidelity-improvements-plan.md` for implementation details
   - API reference: `docs_site/api/services/backtesting/costs.md` (upcoming)